# Date as an input Argument

In [95]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [96]:
# Addapter Layer

def read_csv_to_df(bucket, key, decoding = "utf-8", sep = ","):
    csv_obj = bucket.Object(key = key).get().get("Body").read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter = sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index = False)
    bucket.put_object(Body = out_buffer.getvalue(), Key = key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix = prefix)]
    return files

In [97]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index = True)
    return df
    
def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace = True)
    df["opening_price"] = df.sort_values(by=["Time"]).groupby(["ISIN", "Date"])["StartPrice"].transform("first")
    df["closing_price"] = df.sort_values(by=["Time"]).groupby(["ISIN", "Date"])["StartPrice"].transform("last")
    df = df.groupby(["ISIN", "Date"], as_index = False).agg(opening_price_eur = ("opening_price", "min"),
                                                                closing_price_eur = ("closing_price", "min"), 
                                                                minimum_price_eur = ("MinPrice", "min"),
                                                                maximum_price_eur = ("MaxPrice", "max"),
                                                                daily_traded_volume = ("TradedVolume", "sum"))
    df["prev_closing_price"] = df.sort_values(by = ["Date"]).groupby(["ISIN"])["closing_price_eur"].shift(1)
    df["change_prev_closing_%"] = (df["closing_price_eur"] - df["prev_closing_price"]) / df["prev_closing_price"] * 100
    df.drop(columns = ["prev_closing_price"], inplace = True)
    df = df.round(2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True
    
def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df =  extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

In [98]:
# Application layer - not core

def return_date_list(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days = 1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days = x)).strftime(src_format) for x in range(0, (today - min_date).days + 1)]
    return return_date_list

In [99]:
# main function entry point

def main():
    # Parameters/ Configurations
    # Later read as config
    arg_date = "2022-02-09"
    src_format = "%Y-%m-%d"
    src_bucket = "deutsche-boerse-xetra-pds"
    trg_bucket = "xetra--1234"
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = "xetra_daily_report_"
    trg_format = ".parquet"
    
    # Init
    s3 = boto3.resource("s3")
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application 
    
    date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)
    

In [100]:
# run 
main()

## Read the uploaded file

In [101]:
trg_bucket = "xetra--1234"
s3 = boto3.resource("s3")
bucket_trg = s3.Bucket(trg_bucket)

for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report20220209_205648.parquet
xetra_daily_report20220209_222954.parquet
xetra_daily_report_20220210_143238.parquet
xetra_daily_report_20220210_144413.parquet
xetra_daily_report_20220210_155512.parquet


In [104]:
prq_obj = bucket_trg.Object(key = "xetra_daily_report_20220210_155512.parquet").get().get("Body").read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [105]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-02-09,39.00,38.60,38.50,39.05,905,-0.39
1,AT000000STR1,2022-02-10,38.95,39.45,38.95,39.45,147,2.20
2,AT00000FACC2,2022-02-09,9.48,9.38,9.38,9.49,153,2.29
3,AT00000FACC2,2022-02-10,9.30,9.23,9.22,9.43,620,-1.60
4,AT0000606306,2022-02-09,27.38,28.12,27.32,28.12,1739,1.96
...,...,...,...,...,...,...,...,...
6209,XS2314659447,2022-02-10,8.86,8.96,8.86,8.97,60,1.84
6210,XS2314660700,2022-02-09,20.11,20.47,20.11,20.47,0,2.38
6211,XS2314660700,2022-02-10,20.91,20.80,20.80,20.91,33,1.63
6212,XS2376095068,2022-02-09,38.35,38.30,38.18,38.35,0,0.54
